# Final Project

Molly Siebecker and Marley Myrianthopoulos

CUNY SPS

DATA 620

Summer 2024

In [13]:
# Import required libraries and data
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import itertools

rankings_url = "https://raw.githubusercontent.com/Marley-Myrianthopoulos/cunysps_data620/main/final/nyt_100_books_21st_century.csv"
rankings = pd.read_csv(rankings_url)
print(rankings.head())

   rank                     title            author  \
0     1       My Brilliant Friend    Elena Ferrante   
1     2  The Warmth of Other Suns  Isabel Wilkerson   
2     3                 Wolf Hall     Hilary Mantel   
3     4           The Known World   Edward P. Jones   
4     5           The Corrections  Jonathan Franzen   

                                               blurb   author_blurb  \
0  The first volume of what would become Ferrante...            NaN   
1  Wilkerson’s intimate, stirring, meticulously r...  Dwight Garner   
2  It was hard choosing the books for my list, bu...   Lev Grossman   
3  This novel, about a Black farmer, bootmaker an...  Dwight Garner   
4  With its satirical take on mental health, self...            NaN   

                                    additional_blurb author_additional_blurb  
0                                                NaN                     NaN  
1                                                NaN                     NaN  
2    

In [14]:
ballots_url = "https://raw.githubusercontent.com/Marley-Myrianthopoulos/cunysps_data620/main/final/nyt_100_books_ballots.csv"
ballots = pd.read_csv(ballots_url)
print(ballots.head())

                 voter profession                      book _1  \
0         Stephen King     author                    Atonement   
1          Min Jin Lee     author  All the Light We Cannot See   
2  Karl Ove Knausgaard     author                         2666   
3        Bonnie Garmus     author     Between the World and Me   
4    Curtis Sittenfeld     author           The Line of Beauty   

            author_1                        book _2            author_2  \
0         Ian McEwan                Christine Falls      Benjamin Black   
1      Anthony Doerr  Behind the Beautiful Forevers       Katherine Boo   
2     Roberto Bolano                  The Argonauts       Maggie Nelson   
3   Ta-Nehisi Coates               Demon Copperhead  Barbara Kingsolver   
4  Alan Hollinghurst                    A Lucky Man      Jamel Brinkley   

                   book _3        author_3                  book _4  \
0            The Goldfinch     Donna Tartt                Gone Girl   
1         

In [16]:
print("Columns in DataFrame:", ballots.columns.tolist())


Columns in DataFrame: ['voter', 'profession', 'book _1', 'author_1', 'book _2', 'author_2', 'book _3', 'author_3', 'book _4', 'author_4', 'book _5', 'author_5', 'book _6', 'author_6', 'book _7', 'author_7', 'book _8', 'author_8', 'book _9', 'author_9', 'book _10', 'author_10']


In [17]:
author_to_book = ballots[['voter', 'profession', 'book_1', 'book_2', 'book_3', 'book_4', 'book_5', 'book_6', 'book_7', 'book_8', 'book_9', 'book_10']]

KeyError: "['book_1', 'book_2', 'book_3', 'book_4', 'book_5', 'book_6', 'book_7', 'book_8', 'book_9', 'book_10'] not in index"